In [8]:
import pandas as pd
import numpy as np
import os
from glob import glob
import zipfile

In [9]:
raw_path = 'Data-20250729T201332Z-1-001.zip'

In [10]:
!unzip {raw_path} -d zipped_data

Archive:  Data-20250729T201332Z-1-001.zip
  inflating: zipped_data/Data/How to use data.docx  
  inflating: zipped_data/Data/CRWV.zip  
  inflating: zipped_data/Data/FROG.zip  
  inflating: zipped_data/Data/SOUN.zip  


In [15]:
input_zip_dir = 'zipped_data/Data'
output_extract_dir = 'data'

for path in glob(os.path.join(input_zip_dir, '*.zip')):
    ticker_name = os.path.splitext(os.path.basename(path))[0]
    target_path = os.path.join(output_extract_dir, ticker_name)

    os.makedirs(target_path, exist_ok=True)

    with zipfile.ZipFile(path, 'r') as zip_ref:
        zip_ref.extractall(target_path)

    print(f"Extracted {path} → {target_path}")


Extracted zipped_data/Data/FROG.zip → data/FROG
Extracted zipped_data/Data/CRWV.zip → data/CRWV
Extracted zipped_data/Data/SOUN.zip → data/SOUN


In [ ]:
dfs = []
for 